In [1]:
#########################################################################
#Data Collection 
#
# Functions for handling subwatershed characteristics data collection
#
# Authors: Shannon McAvoy (smcavoy@dewberry.com)
#
# Editor: Evan Barnes (ebarnes@dewberry.com)
#
# Copyright: Dewberry Engineers Inc.
#########################################################################

In [2]:
import pandas as pd
import arcpy
import numpy as np
from numpy import mean
from numpy import std
import os
from arcpy import management
# from arcgis.gis import GIS
from time import time
arcpy.CheckOutExtension('Spatial')
# import rasterio

'CheckedOut'

In [3]:
arcpy.env.overwriteOutput = True

In [4]:
path = "C:\\Users\jmatney\Documents\GitHub\IndianaRisk\data\Indiana_ML"

In [5]:
hucs = "C:\\Users\jmatney\Documents\GitHub\IndianaRisk\data\HUCs"

In [6]:
st = time()#
# Set Local Variables
output_workspace = os.path.join(path, "working_dsn")

arcpy.env.workspace = output_workspace

##### change these based on what subwatersheds you want #####

#set source for shapefile with all 11 subwatersheds in the county

# all_subwatersheds = r"P:\Temp\McAvoy\ML_DataCollection\Indiana_ML\Indiana_HUC12_groups\1.shp"
all_subwatersheds = os.path.join(hucs, "Final_Fix.shp")

#set source for streams data
streams = os.path.join(path, "Indiana_50c_streams_edited\Indiana_50c_streams_edited.shp")

#####these are the same for all subwatersheds in Indiana#####

#set source for dem
dem_Indiana = os.path.join(path, "IN_EXTENT_MOSAIC\IN_EXTENT_MOSAIC.tif")

#set source for slope dem
dem_slope = os.path.join(path, "Indiana_SlopeRaster\IN_slope2")

#set source for NHFL Data
nfhl_sfha = os.path.join(path, "NFHL_18_20200310.gdb\S_FLD_HAZ_AR")

#set source for water bodies data 
water_bodies = os.path.join(path, "IndianaMAP_WaterBodies\Water_Bodies_Lakes_LocalRes\Hydrography_LocalRes_WaterbodyDiscrete_NHD_IN.shp")

#set source for dams data 
dams = os.path.join(path, "IndianaMAP_Dams\Dams_IDNR\Dams_IDNR_IN.shp")

#set source for bridges data
bridges = os.path.join(path, "IndianaMAP_Bridges\Bridges_County_INDOT\Bridges_County_INDOT_IN.shp")

#set source for streets data
streets = os.path.join(path, "IndianaMAP_Streets\Streets_Centerlines_IGIO\County_Street_Centerlines_IGIO_IN.gdb\County_Street_Centerlines_IGIO_IN_Dec2019")

#set source for railraods data
railroads = os.path.join(path, "IndianaMAP_Railroads\Railroads_Active_Abandoned_INDOT\Rail_System_Active_Abandoned_INDOT_IN.shp")

#set source for ACS population data
population = os.path.join(path, "Indiana_PopulationData\Indiana_PopulationData.shp")

#set source for ACS median income data 
median_income = os.path.join(path, "Indiana_IncomeData\Indiana_IncomeData.shp")

#set source for county boundary data
county_boundary = os.path.join(path, "MarionCounty_Boundary\Marion_County_Boundary.shp")

#source for Bing building footprints for Indiana (attrubuted with open street maps data)
building_footprints = os.path.join(path, "BuildingFootprints_Indiana\Building_Footprints_Attributed_IN.shp")

#folder with partial duration files, set as workspace temporarily, then reset when done
directory_rainfall = os.path.join(path, "MarionCounty_Rainfall\All_Rainfall_Clipped_IN")

#set source for nlcd land use data
lu_usa = os.path.join(path, r"NLCD_Impervious\NLCD_indiana_polygon.shp")

#set source for impervious indicator data
impervious_usa = os.path.join(path, "NLCD_Impervious\\NLCD_2016_Impervious_L48_20190405_PERCENT\\NLCD_2016_Impervious_L48_20190405.img")


#########################################################################################################
print(round(((time()-st)/60), 2) , 'minutes to process.')

0.01 minutes to process.


In [7]:
subwatershed_list = []
area_list = []
perimeter_list = []
watershed_length_list = []
elongation_ratio_list = []
shape_factor_list = []
circulatory_ratio_list = []
relief_list = []
relief_ratio_list = []
avg_slope_list = []
drainage_density_list = []
ruggedness_list = []
aae_list = []
buildings_aae_list = []
x_list = []
buildings_x_list = []
water_bodies_list = []
dams_list = []
bridges_list = []
streets_list = []
railroads_list = []
population_list = []
dependent_population_list = []
population_density_list = []
avg_median_income_list = []
housing_density_list = []
population_change_list = []
dist_to_stream_avg_list = []
dist_to_stream_stdev_list = []

lu_21_list = []
lu_22_list = []
lu_23_list = []
lu_24_list = []
lu_41_list = []
lu_82_list = []
impervious_percent_list = []

# orb100yr06h_list = []
# orb100yr12h_list = []
# orb100yr24h_list = []
# orb25yr06h_list = []
# orb25yr12h_list = []
# orb25yr24h_list = []
# orb2yr06h_list = []
# orb2yr12h_list = []
# orb2yr24h_list = []
# orb50yr06h_list = []
# orb50yr12h_list = []
# orb50yr24h_list = []
# orb100yr06ha_am_list = []
# orb100yr12ha_am_list = []
orb100yr24ha_am_list = []
# orb25yr06ha_am_list = []
# orb25yr12ha_am_list = []
# orb25yr24ha_am_list = []
# orb2yr06ha_am_list = []
# orb2yr12ha_am_list = []
# orb2yr24ha_am_list = []
# orb50yr06ha_am_list = []
# orb50yr12ha_am_list = []
# orb50yr24ha_am_list = []

In [8]:
print(all_subwatersheds)

C:\Users\jmatney\Documents\GitHub\IndianaRisk\data\HUCs\Final_Fix.shp


In [9]:
#add AREA and PERIMETER fields for the whole subwatersheds file
#calculate area of subwatersheds
arcpy.AddField_management(all_subwatersheds, "AREA", "DOUBLE")

arcpy.CalculateGeometryAttributes_management(all_subwatersheds, "AREA AREA_GEODESIC", '', 
                                                    "SQUARE_KILOMETERS",
                                                    None)
#calculate perimeter of subwatershed
arcpy.AddField_management(all_subwatersheds, "PERIMETER", "DOUBLE")
perimeter = arcpy.CalculateGeometryAttributes_management(all_subwatersheds, "PERIMETER PERIMETER_LENGTH_GEODESIC",
                                                         "KILOMETERS")

print("perimeter calculated")

perimeter calculated


In [10]:
st = time()
# perform all need intersections here:

#put all intersections in this projection so area and length will be in meters
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(102008)

nfhl_sfha_intersect = "nfhl_sfha.shp"
nfhl_sfha_intersect = arcpy.Intersect_analysis([nfhl_sfha, all_subwatersheds], nfhl_sfha_intersect)

water_bodies_intersect = "water_bodies.shp"
water_bodies_intersect = arcpy.Intersect_analysis([water_bodies, all_subwatersheds], water_bodies_intersect)

dams_intersect = "dams.shp"
arcpy.Intersect_analysis([dams, all_subwatersheds], dams_intersect)

bridges_intersect = "bridges.shp"
arcpy.Intersect_analysis([bridges, all_subwatersheds], bridges_intersect)

streets_intersect = "streets.shp"
arcpy.Intersect_analysis([streets, all_subwatersheds], streets_intersect)

railroads_intersect = "railroads.shp"
arcpy.Intersect_analysis([railroads, all_subwatersheds], railroads_intersect)

population_intersect = "population.shp"
arcpy.Intersect_analysis([population, all_subwatersheds], population_intersect)

median_income_intersect= "median_income.shp"
arcpy.Intersect_analysis([median_income, all_subwatersheds], median_income_intersect)

building_footprints_intersect = "building_footprints.shp"
arcpy.Intersect_analysis([building_footprints, all_subwatersheds], building_footprints_intersect)

streams_intersect = "streams.shp"
arcpy.Intersect_analysis([streams, all_subwatersheds], streams_intersect)

lu_subwatershed = "lu_subwatershed.shp"
arcpy.Intersect_analysis([lu_usa, all_subwatersheds], lu_subwatershed)

print(round(((time()-st)/60), 2) , 'minutes to process.')

5.41 minutes to process.


In [11]:
def get_length_area(layer, length_area):
    #length_Area = 'LENGTH' or 'AREA'
    search_field = "SHAPE@{}".format(length_area)
    values = []
    with arcpy.da.SearchCursor(layer, [search_field]) as sCursor: 
        for s in sCursor:
            # convert meters to kilometers
            if length_area == 'AREA':
                value = s[0]/ (1000 * 1000)
            else: # assumed length
                value = s[0] / 1000
                    
            values.append(value)
    return values

In [12]:
st = time()
# make all layers for selections
subwatershed_selection = "subwatershed_lyr"
arcpy.MakeFeatureLayer_management (all_subwatersheds, subwatershed_selection)

nfhl_sfha_selection = "nfhl_sfha_lyr"
arcpy.MakeFeatureLayer_management(nfhl_sfha_intersect, nfhl_sfha_selection)

water_bodies_selection = "water_bodies_lyr"
arcpy.MakeFeatureLayer_management(water_bodies_intersect, water_bodies_selection)

dams_selection = "dams_lyr"
arcpy.MakeFeatureLayer_management(dams_intersect, dams_selection)

bridges_selection = "bridges_lyr"
arcpy.MakeFeatureLayer_management(bridges_intersect, bridges_selection)

streets_selection = "streets_lyr"
arcpy.MakeFeatureLayer_management(streets_intersect, streets_selection)

railroads_selection = "railroads_lyr"
arcpy.MakeFeatureLayer_management(railroads_intersect, railroads_selection)

population_selection = "population_lyr"
arcpy.MakeFeatureLayer_management(population_intersect, population_selection)

median_income_selection = "median_income_lyr"
arcpy.MakeFeatureLayer_management(median_income_intersect, median_income_selection)

building_footprints_selection= "building_footprints_lyr"
arcpy.MakeFeatureLayer_management(building_footprints_intersect, building_footprints_selection)

streams_selection = "streams_lyr"
arcpy.MakeFeatureLayer_management(streams_intersect, streams_selection)

lu_selection = "lu_lyr"
arcpy.MakeFeatureLayer_management(lu_subwatershed, lu_selection)

print(round(((time()-st)/60), 2) , 'minutes to process.')

0.37 minutes to process.


In [13]:
# #set source for dem
# dem_Indiana = os.path.join(path, "IN_EXTENT_MOSAIC\IN_EXTENT_MOSAIC.tif")
all_subwatersheds

'C:\\Users\\jmatney\\Documents\\GitHub\\IndianaRisk\\data\\HUCs\\Final_Fix.shp'

In [14]:
st = time()


#search cursor through each row of county subwatersheds file
with arcpy.da.SearchCursor(all_subwatersheds, ['HUC12','AREA', 'PERIMETER']) as cursor:

    for row in cursor:
        if row[0] in ['051202081004']:
            next(cursor) # Does nothing, skips next item
        else:
            subwatershed_number = row[0]

            subwatershed_list.append(subwatershed_number)

            print("--------------------", subwatershed_number, " is the subwatershed to be worked on ------------------------")
            # Make a layer from the feature class
            arcpy.MakeFeatureLayer_management(all_subwatersheds,"subwatersheds_lyr")

            #select subwatershed from shapefile with all of them
            s = arcpy.SelectLayerByAttribute_management(all_subwatersheds, "NEW_SELECTION", 
                                                    "HUC12 = '{}'".format(subwatershed_number))
            arcpy.CopyFeatures_management(s, "s_lyr.shp")
            subwatershed_selection1 = os.path.join(path, "working_dsn\s_lyr.shp")
            #arcpy.MakeFeatureLayer_management(s, subwatershed_selection1)

            # use mask in lieu of clipping rasters
            arcpy.env.cellSize = dem_slope
            arcpy.env.snapRaster = dem_slope
            #arcpy.env.outputCoordinateSystem = 
            arcpy.env.mask = subwatershed_selection1

            area = row[1]
            area_list.append(area)
            print(area, " square kilometers")

            perimeter = row[2]
            perimeter_list.append(perimeter)
            print(perimeter, " kilometers")

            #get slope value
            avg_slope_zonalst = arcpy.gp.ZonalStatistics_sa(subwatershed_selection1, "FID", dem_slope, "MEAN" )
            avg_slope_result = arcpy.GetRasterProperties_management(avg_slope_zonalst, "MAXIMUM")
            avg_slope_value = avg_slope_result.getOutput(0)
            avg_slope_list.append(avg_slope_value)
            print(avg_slope_value, " = avg slope (%)")


            #calculate circulatory ratio
            #ratio of area to the area of a circle having equal perimeter as the perimeter of drainage basin

            #area of a circle with same perimeter as above
            #C = 2(pi)r
            #r = C/(2pi)
            #A = (pi)r^2
            circle_radius = perimeter/(2*np.pi)
            circle_area = (np.pi*(circle_radius**2))

            #ratio of subwatershed perimeter to circle circumference
            circulatory_ratio = area / circle_area

            circulatory_ratio_list.append(circulatory_ratio)
            print(circulatory_ratio, " is the circulatory ratio")


            #calculate relief
            #elevation difference before basin outlet and highest point located in the perimeter of basin

            #find highest point on perimeter

            #use Raster Domain tool to get z-enabled polyline of perimeter
            dem_clip =  os.path.join(path, "working_dsn\dem_clip.tif")
            arcpy.Clip_management(dem_Indiana, "#", dem_clip, subwatershed_selection1 )
            perimeter_polyline = "perimeter_polyline.shp"
            arcpy.RasterDomain_3d(dem_clip, perimeter_polyline, "LINE")
            print("created 3d polyline of subwatershed perimeter")

            #Convert each vertices of the polyline into points
            perimeter_points = "perimeter_points.shp"
            arcpy.FeatureVerticesToPoints_management(perimeter_polyline, perimeter_points)
            print("created perimeter points")

            #get Z values into the attribute table for the points
            arcpy.AddZInformation_3d(perimeter_points, "Z")
            print("added z information")

            #get the max value from all of the points
            perimeter_stats = "perimeter_stats"
            arcpy.Statistics_analysis(perimeter_points, "perimeter_stats", [["Z", "MAX"], ["Z", "MIN"]])

            #get the values for max Z and min Z into a format to use them
            with arcpy.da.SearchCursor(perimeter_stats, ['MAX_Z', 'MIN_Z']) as statsCursor:
                for stat in statsCursor:
                    max_z = stat[0]
                    min_z = stat[1]


            print(max_z)
            print(min_z)

            relief = max_z - min_z
            print("The relief of the subwatershed is: ", relief," meters.")

            relief_list.append(relief)

            #find area covered by A and AE zones and X NFHL Zones

            #select Zone A and Zone AE
            arcpy.management.SelectLayerByAttribute(nfhl_sfha_selection, "NEW_SELECTION", "HUC12 = '{}'".format(subwatershed_number))

            #get the sum of all of the areas
            aae_area_list = []  
            x_area_list = []
            with arcpy.da.SearchCursor(nfhl_sfha_selection, ['SHAPE@AREA', 'FLD_ZONE', 'ZONE_SUBTY']) as selCursor : 
                for sel in selCursor:
                    fld_zone = sel[1]
                    zone_subty = sel[2]
                    area = sel[0]/(1000*1000)
                    if fld_zone == 'AE' or fld_zone == 'A':
                        aae_area_list.append(area)
                    if fld_zone == 'X' and zone_subty == '0.2 PCT ANNUAL CHANCE FLOOD HAZARD':
                        x_area_list.append(area)


            aae_area_sum = sum(aae_area_list)
            aae_list.append(aae_area_sum)

            x_area_sum = sum(x_area_list)
            x_list.append(x_area_sum)
            print(x_area_sum, " = area of all X zones, 0.2PCT zone subtype in the subwatershed (square km)")


            #calculate area covered by lakes/reserviors 
            arcpy.management.SelectLayerByAttribute(water_bodies_selection, "NEW_SELECTION", 
                                                                "HUC12 = '{}'".format(subwatershed_number))
            water_bodies_area_sum = sum(get_length_area(water_bodies_selection, "AREA"))
            water_bodies_list.append(water_bodies_area_sum)
            print(water_bodies_area_sum, " = area of all water bodies in the subwatershed (square km)")


            #count all of the dams in the subwatershed
            arcpy.management.SelectLayerByAttribute(dams_selection, "NEW_SELECTION", 
                                                                "HUC12 = '{}'".format(subwatershed_number))
            #count number of dam points
            dams_count = arcpy.GetCount_management(dams_selection)
            dams_list.append(dams_count)
            print(dams_count, " = number of dams in the subwatershed")


            #count all of the bridges in the subwatershed
            arcpy.management.SelectLayerByAttribute(bridges_selection, "NEW_SELECTION", 
                                                                "HUC12 = '{}'".format(subwatershed_number))
            #count number of dam points
            bridges_count = arcpy.GetCount_management(bridges_selection)
            bridges_list.append(bridges_count)
            print(bridges_count, " = number of bridges in the subwatershed")

            #calculate the kilometers of streets in the subwatershed
            arcpy.management.SelectLayerByAttribute(streets_selection, "NEW_SELECTION", 
                                                                "HUC12 = '{}'".format(subwatershed_number))
            streets_length_sum = sum(get_length_area(streets_selection, "LENGTH"))
            streets_list.append(streets_length_sum)
            print(streets_length_sum, " = sum of all streets in the subwatershed (km)")


            #calculate km of railroads in subwatershed
            arcpy.management.SelectLayerByAttribute(railroads_selection, "NEW_SELECTION", 
                                                                "HUC12 = '{}'".format(subwatershed_number))
            railroads_length_sum = sum(get_length_area(railroads_selection, "LENGTH"))
            railroads_list.append(railroads_length_sum)
            print(railroads_length_sum, " = sum of all railroads in the subwatershed (km)")   


            #ACS population data - 5 year estimates (2014-2018) gotten from ESRI Living Atlas Data
            #data is chosen to be on census tract level
            arcpy.management.SelectLayerByAttribute(population_selection, "NEW_SELECTION", 
                                                                "HUC12 = '{}'".format(subwatershed_number))
            #get total population
            #field = B01001_001E = Total Population (alias)
            #when clipped, field name changes to B01001_001

            #get the sum of all of the populations in each tract
            total_pop_list = []  
            with arcpy.da.SearchCursor(population_selection, ['B01001_001']) as popCursor:  
                for pop in popCursor:  
                    total_pop_value = pop[0] 
                    total_pop_list.append(total_pop_value)  
            total_pop_sum = sum(total_pop_list)
            population_list.append(total_pop_sum)
            print(total_pop_sum, " = total population in the subwatershed") 

            #get the average of percentages of dependent age groups in each tract
            #field = B01001_calc_pctDependE = Percent of Population in Dependent Age Groups (under 18 and 65+) (alias)
            #when clipped field changes to B01001_61
            dependent_pop_list = []  
            with arcpy.da.SearchCursor(population_selection, ["B01001__61"]) as dpopCursor: 
                for dpop in dpopCursor:  
                    dependent_pop_value = dpop[0] 
                    dependent_pop_list.append(dependent_pop_value)  
            dependent_pop_avg_pct = mean(dependent_pop_list)
            dependent_population_list.append(dependent_pop_avg_pct)

            print(dependent_pop_avg_pct, " = total percent of dependent population in the subwatershed") 

            #find population density
            population_density = total_pop_sum / area
            population_density_list.append(population_density)
            print(population_density, " = population density of subwatershed (people/square km)")


            #get average median income 
            arcpy.management.SelectLayerByAttribute(median_income_selection, "NEW_SELECTION", 
                                                                "HUC12 = '{}'".format(subwatershed_number))

            #field = B19049_001 = Median Household Income in past 12 months 
            #^(inflation-adjusted dollars to last year of 5-year range) (alias)
            #get the sum of all of the populations in each tract
            total_median_income_list = []  
            with arcpy.da.SearchCursor(median_income_selection, ['B19049_001']) as incomeCursor:  
                for income in incomeCursor:  
                    median_income_value = income[0] 
                    total_median_income_list.append(median_income_value)  
            median_income_average = mean(total_median_income_list)
            avg_median_income_list.append(median_income_average)
            print(median_income_average, " = average median income in the subwatershed") 


            #get housing density
            #select buildings that are marked residential
            arcpy.SelectLayerByAttribute_management(building_footprints_selection, "NEW_SELECTION",
                                                                                 "HUC12 = '{}' And RES_NONRES = 'Res'".format(subwatershed_number))
            #get count of how many buildings there are 
            buildings_count = arcpy.GetCount_management(building_footprints_selection)
            buildings_count_number = buildings_count.getOutput(0)
            print(buildings_count_number, " = number of residential building footprints in the subwatershed")
            #divide number of buildings by subwatershed area
            housing_density = int(buildings_count_number) / area
            housing_density_list.append(housing_density)
            print(housing_density, " = housing density (buildings per square km)")


           #find total population from the 2013 5-year ACS estimates
            #get the sum of all of the 2013 populations in each tract
            #field = DP05_0001E = TotalPopulation from the 2013 ACS 5yr estimates
            total_pop_list_2013 = []  

            with arcpy.da.SearchCursor(population_selection, ['DP05_0001E']) as oldpopCursor:  
                for oldpop in oldpopCursor:  
                    total_pop_value_2013 = oldpop[0]
                    if total_pop_value_2013 != '0':
                        total_pop_list_2013.append(int(total_pop_value_2013))  

            total_pop_sum_2013 = sum(total_pop_list_2013)

            print(total_pop_sum_2013, " = total 2013 population in the subwatershed") 

            #find population change between 2018 and 2013
            population_change = total_pop_sum - total_pop_sum_2013

            population_change_list.append(population_change)
            print(population_change, " = population change between 2013 and 2018")


            #calculate drainage density
            #the total length of all streams and tributaries divided by basin area

            #all streams in subwatershed
            arcpy.management.SelectLayerByAttribute(streams_selection, "NEW_SELECTION", 
                                                                "HUC12 = '{}'".format(subwatershed_number))
            print("streams clipped")
            print("length calculated")
            #get the sum of all of the areas
            stream_length_list = get_length_area(streams_selection, "LENGTH")  
            stream_length_sum = sum(stream_length_list)
            print(stream_length_sum, " = length of all streams in subwatershed")
            print(area, " = subwatershed area")
            drainage_density = stream_length_sum / area
            drainage_density_list.append(drainage_density)
            print(drainage_density, "is the drainage density (streams/km)")
            #find watershed length
            #watershed length = distance from outlet to watershed boundary along the main channel
            #we are assuming that the longest stream above is the main channel
            watershed_length = max(stream_length_list)
            watershed_length_list.append(watershed_length)
            print(watershed_length, " = watershed length (longest stream in subwatershed)")
            #calculate shape factor 
            #watershed length squared divided by watershed area
            shape_factor = (watershed_length**2) / area
            shape_factor_list.append(shape_factor)
            print(shape_factor, " is the shape factor.")


            #calculate relief ratio
            #relief divided by watershed length
            #length is is kilometers, convert to meters
            watershed_length_meters = watershed_length * 1000
            relief_ratio = relief / watershed_length_meters
            relief_ratio_list.append(relief_ratio)
            print(relief_ratio, " = relief ratio")


            #calculate ruggedness number
            #product of relief and drainage density
            #relief is in meters, convert first to km
            relief_km = relief / 1000
            print(relief_km, " is the relief in km")
            ruggedness = relief_km * drainage_density
            ruggedness_list.append(ruggedness)
            print(ruggedness, " is the ruggedness number")


            #calculate elongation ratio
            #ratio of diameter of a circle having the same area as the basin to the max basin length


            #diameter of circle with same area
            #A = (pi)r^2
            #r = sqrt(A/pi)
            #d = r*2

            radius = np.sqrt(area/np.pi)
            print(radius, " miles is the radius of a circle with the same area.")
            diameter = radius*2
            print(diameter, " miles is the diameter of a circle with the same area.")
            elongation_ratio = diameter / watershed_length
            elongation_ratio_list.append(elongation_ratio)


            #get number of buildings inside the aae zone
            arcpy.SelectLayerByAttribute_management(building_footprints_selection, "NEW_SELECTION",
                                                                                 "HUC12 = '{}'".format(subwatershed_number))
            arcpy.management.SelectLayerByAttribute(nfhl_sfha_selection, "NEW_SELECTION", 
                                                                "HUC12 = '{}' And (FLD_ZONE = 'AE' Or FLD_ZONE = 'A')".format(subwatershed_number))
            arcpy.SelectLayerByLocation_management(building_footprints_selection, "INTERSECT", nfhl_sfha_selection, selection_type = 'SUBSET_SELECTION' )
            buildings_aae_count = arcpy.GetCount_management(building_footprints_selection)
            buildings_aae_list.append(buildings_aae_count)

            #get number of buildings inside the x zone
            arcpy.SelectLayerByAttribute_management(building_footprints_selection, "NEW_SELECTION",
                                                                                 "HUC12 = '{}'".format(subwatershed_number))
            arcpy.management.SelectLayerByAttribute(nfhl_sfha_selection, "NEW_SELECTION", 
                                                               "HUC12 = '{}' And FLD_ZONE = 'X' And ZONE_SUBTY = '0.2 PCT ANNUAL CHANCE FLOOD HAZARD'".format(subwatershed_number))
            arcpy.SelectLayerByLocation_management(building_footprints_selection, "INTERSECT", nfhl_sfha_selection, selection_type = 'SUBSET_SELECTION' )
            buildings_x_count = arcpy.GetCount_management(building_footprints_selection)
            buildings_x_list.append(buildings_x_count)

            #get area of various land use codes
            #clip usa land use polygon to the subwatershed


            #get percent impervious indicator for subwatershed area
            #get avg value  value
            arcpy.env.cellSize = impervious_usa
            arcpy.env.snapRaster = impervious_usa
            #arcpy.env.outputCoordinateSystem = 
            arcpy.env.mask = subwatershed_selection1

            avg_impervious_indicator = arcpy.sa.ZonalStatistics(subwatershed_selection1, "FID", impervious_usa, "MEAN" )

            avg_impervious_result = arcpy.GetRasterProperties_management(avg_impervious_indicator, "MAXIMUM")

            avg_impervious_pct_value = avg_impervious_result.getOutput(0)
            print(avg_impervious_pct_value, " = avg impervious percent ")

            impervious_percent_list.append(avg_impervious_pct_value)


            #get distance from residential buildings to streams
            #project buildings and streams so they are in the same GCS
            output_coord_system = arcpy.SpatialReference(os.path.join(path, "working_dsn\\NAD1983_ProjectionFile.prj"))
            streams_project = "streams_project.shp"
            arcpy.Project_management(streams_selection, streams_project, output_coord_system)
            building_footprints_residential_copy = "buildings_res_copy.shp"
            arcpy.CopyFeatures_management(building_footprints_selection, building_footprints_residential_copy)
            buildings_project = "buildings_res_project.shp"
            arcpy.Project_management(building_footprints_residential_copy, buildings_project, output_coord_system)

            #use near tool to get distance to steams
            arcpy.Near_analysis(buildings_project, streams_project, "", "LOCATION", "", "GEODESIC")

            #get all the distances in to the streams
            dist_to_stream_list = []
            with arcpy.da.SearchCursor(buildings_project, ["NEAR_DIST"]) as distCursor: 
                for dist in distCursor:  
                    dist_to_stream = dist[0]  
                    dist_to_stream_list.append(dist_to_stream)  

            dist_to_stream_avg = mean(dist_to_stream_list)

            dist_to_stream_stdev = std(dist_to_stream_list)

            print(dist_to_stream_avg)
            print(dist_to_stream_stdev)

            dist_to_stream_avg_list.append(dist_to_stream_avg)
            dist_to_stream_stdev_list.append(dist_to_stream_stdev)


            ## LOOP    

            #loop through all rainfall rasters in the same folder 


            arcpy.env.workspace = directory_rainfall


            rasters = arcpy.ListRasters()

            for raster in rasters:

                arcpy.env.cellSize = raster
                arcpy.env.snapRaster = raster

                raster_name = os.path.basename(raster).rstrip(os.path.splitext(raster)[1])

                to_process = [
                    "orb100yr24ha_am"
                ]

                if raster_name in to_process:
                    print(raster_name)


                    #get average rainfall for duration using zonal statistics
                    avg_rainfall = arcpy.sa.ZonalStatistics(subwatershed_selection1, "FID", raster, "MEAN" )

                    #get avg value value
                    avg_result = arcpy.GetRasterProperties_management(avg_rainfall, "MAXIMUM")
                    rainfall_value = avg_result.getOutput(0)
                    print(rainfall_value, " = avg rainfall in inches *1000")


                    if raster_name == "orb100yr24ha_am":
                        orb100yr24ha_am_list.append(rainfall_value)

                    else:
                        continue 



            #set workspace environment back to the newly created folder
            arcpy.env.workspace = output_workspace
            print(round(((time()-st)/60), 2) , 'minutes to process.')




        #########################################################################################




    print(round(((time()-st)/60), 2) , 'minutes to process.')

-------------------- 051202060310  is the subwatershed to be worked on ------------------------
94.9985030981  square kilometers
56.6302129052  kilometers
4.84825944900513  = avg slope (%)
0.3722461694174872  is the circulatory ratio
created 3d polyline of subwatershed perimeter
created perimeter points
added z information
255.0
0.0
The relief of the subwatershed is:  255.0  meters.
0.39955064482946445  = area of all X zones, 0.2PCT zone subtype in the subwatershed (square km)
0.6510598905016047  = area of all water bodies in the subwatershed (square km)
0  = number of dams in the subwatershed
20  = number of bridges in the subwatershed
147.90008707066917  = sum of all streets in the subwatershed (km)
8.689067961591023  = sum of all railroads in the subwatershed (km)
23299  = total population in the subwatershed
37.9  = total percent of dependent population in the subwatershed
15395385254.763136  = population density of subwatershed (people/square km)
59298.0  = average median income i

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 010568: Invalid extent. Please check for zero length or width, or failure to project extent to output spatial reference. 
Failed to execute (ZonalStatistics).


In [17]:
st = time()
outputs = {'subwatershed': subwatershed_list[0:11], # 
          'area': area_list[0:11],
          'perimeter': perimeter_list[0:11],
          'circulatory_ratio': circulatory_ratio_list[0:11], #
          'relief': relief_list[0:11], # 
          'avg_slope': avg_slope_list[0:11], # 
           'watershed_length': watershed_length_list[0:11],
           'elongation_ratio': elongation_ratio_list[0:11], # 
           'drainage_density': drainage_density_list[0:11], # 
           'shape_factor': shape_factor_list[0:11], # 
           'relief_ratio': relief_ratio_list[0:11], # 
           'ruggedness': ruggedness_list[0:11], # 
           'aae_area': aae_list[0:11], # 
           'buildings_aae_count': buildings_aae_list[0:11], # 
           'x_area': x_list[0:11],
           'buildings_x_count': buildings_x_list[0:11], # 
           'water_bodies_area': water_bodies_list[0:11], # 
           'dams_count': dams_list[0:11], # 
           'bridges_count': bridges_list[0:11], # 
           'streets_km': streets_list[0:11], # 
           'railroads_km': railroads_list[0:11], # 
           'population': population_list[0:11], 
           'population_density': population_density_list[0:11], # 
           'avg_median_income': avg_median_income_list[0:11], # 
           'housing_density': housing_density_list[0:11],
           'population_change': population_change_list[0:11], # 
           'dependent_population_pct': dependent_population_list[0:11], # 
           'dist_to_stream_avg (m)': dist_to_stream_avg_list[0:11], # 
           'dist_to_stream_stdev (m)': dist_to_stream_stdev_list[0:11], # 
#            'lu_21_area' : lu_21_list,
#            'lu_22_area' : lu_22_list,
#            'lu_23_area' : lu_23_list,
#            'lu_24_area': lu_24_list,
#            'lu_41_area': lu_41_list,
#            'lu_82_area': lu_82_list,
           'avg_impervious_percent': impervious_percent_list[0:11], # 

           'orb100yr24ha_am': orb100yr24ha_am_list[0:11], # 

          }

            
outputs_df = pd.DataFrame(outputs, columns = ['subwatershed',
                                             'area',
                                             'perimeter',
                                             'circulatory_ratio',
                                             'relief',
                                             'avg_slope',
                                              'watershed_length',
                                              'elongation_ratio',
                                              'drainage_density',
                                              'shape_factor',
                                              'relief_ratio',
                                              'ruggedness',
                                              'aae_area',
                                              'buildings_aae_count',
                                              'x_area',
                                              'buildings_x_count',
                                              'water_bodies_area',
                                              'dams_count',
                                              'bridges_count',
                                              'streets_km',
                                              'railroads_km',
                                              'population',
                                              'population_density',
                                              'avg_median_income',
                                              'housing_density',
                                              'population_change',
                                              'dependent_population_pct',
                                              'dist_to_stream_avg (m)',
                                              'dist_to_stream_stdev (m)',
#                                               'lu_21_area',
#                                               'lu_22_area',
#                                               'lu_23_area',
#                                               'lu_24_area',
#                                               'lu_41_area',
#                                               'lu_82_area',
                                              'avg_impervious_percent',

                                              'orb100yr24ha_am',

                                             ])    
    

#     
#  
# 
# 
# 
#




print(outputs_df)

outputs_df.to_excel(os.path.join(path,"RESULTS\\RESULTS_June9_5.xlsx"))
print(round(((time()-st)/60), 2) , 'minutes to process.')

    subwatershed       area  perimeter  circulatory_ratio      relief  \
0   050800030601  55.950870  36.740381           0.520870  255.000000   
1   051202050303  72.516261  51.452327           0.344219  255.000000   
2   051202011004  57.152624  50.713454           0.279254  255.000000   
3   051202011104  75.846028  47.473964           0.422894  255.000000   
4   051201060204  69.785858  57.117909           0.268802  255.000000   
5   071200030303  41.878013  43.515603           0.277911  242.000000   
6   051202081004  34.574301  43.704360           0.227465  255.000000   
7   050800030601  55.950870  36.740381           0.520870  255.000000   
8   051202050303  72.516261  51.452327           0.344219  255.000000   
9   051202011004  57.152624  50.713454           0.279254  255.000000   
10  051202011104  75.846028  47.473964           0.422894  255.000000   
11  051201060204  69.785858  57.117909           0.268802  255.000000   
12  071200030303  41.878013  43.515603           0.

In [16]:
len(orb100yr24ha_am_list)

14

In [ ]:
orb100yr24ha_am_list[0:13]